# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from scraper import fetch_website_links, fetch_website_contents
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-5-nano'
openai = OpenAI()

API key looks good so far


In [3]:
links = fetch_website_links("https://edwarddonner.com")
links

['https://edwarddonner.com/',
 'https://edwarddonner.com/curriculum/',
 'https://edwarddonner.com/proficient/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://edwarddonner.com/curriculum/',
 'https://edwarddonner.com/2026/02/17/ai-coder-vibe-coder-to-agentic-engineer/',
 'https://edwarddonner.com/2026/02/17/ai-coder-vibe-coder-to-agentic-engineer/',
 'https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/',
 'https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/',
 'https://edwarddonner.com/2025/11/11/ai-live-event/',
 'https://edwarddonner.co

## First step: Have GPT-5-nano figure out which links are relevant

### Use a call to gpt-5-nano to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [14]:
link_system_prompt = """
You are provided with a list of links found on a webpage.
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:

{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [16]:
def get_links_user_prompt(url):
    user_prompt = f"""
Here is the list of links on the website {url} -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

"""
    links = fetch_website_links(url)
    user_prompt += "\n".join(links)
    return user_prompt

In [19]:
print(get_links_user_prompt("https://edwarddonner.com"))


Here is the list of links on the website https://edwarddonner.com -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

https://edwarddonner.com/
https://edwarddonner.com/curriculum/
https://edwarddonner.com/proficient/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://edwarddonner.com/curriculum/
https://edwarddonner.com/2026/02/17/ai-coder-vibe-coder-to-agentic-engineer/
https://edwarddonner.com/2026/02/17/ai-coder-vibe-coder-to-agentic-engineer/
https://edwarddonner.

In [12]:
def select_relevant_links(url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    return links
    

In [20]:
select_relevant_links("https://edwarddonner.com")

{'links': [{'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'company page',
   'url': 'https://nebula.io/?utm_source=ed&utm_medium=referral'}]}

In [21]:
def select_relevant_links(url):
    print(f"Selecting relevant links for {url} by calling {MODEL}")
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    print(f"Found {len(links['links'])} relevant links")
    return links

In [22]:
select_relevant_links("https://edwarddonner.com")

Selecting relevant links for https://edwarddonner.com by calling gpt-5-nano
Found 6 relevant links


{'links': [{'type': 'home page', 'url': 'https://edwarddonner.com/'},
  {'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'company page',
   'url': 'https://nebula.io/?utm_source=ed&utm_medium=referral'},
  {'type': 'linkedin profile', 'url': 'https://www.linkedin.com/in/eddonner/'},
  {'type': 'twitter profile', 'url': 'https://twitter.com/edwarddonner'},
  {'type': 'facebook page',
   'url': 'https://www.facebook.com/edward.donner.52'}]}

In [23]:
select_relevant_links("https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 8 relevant links


{'links': [{'type': 'homepage', 'url': 'https://huggingface.co/'},
  {'type': 'about page', 'url': 'https://huggingface.co/brand'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'code repository', 'url': 'https://github.com/huggingface'},
  {'type': 'linkedin page',
   'url': 'https://www.linkedin.com/company/huggingface/'},
  {'type': 'blog page', 'url': 'https://huggingface.co/blog'},
  {'type': 'product page', 'url': 'https://endpoints.huggingface.co'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT-5-nano

In [27]:
def fetch_page_and_all_relevant_links(url):
    contents = fetch_website_contents(url)
    relevant_links = select_relevant_links(url)
    result = f"## Landing Page:\n\n{contents}\n## Relevant Links:\n"
    for link in relevant_links['links']:
        result += f"\n\n### Link: {link['type']}\n"
        result += fetch_website_contents(link["url"])
    return result

In [28]:
print(fetch_page_and_all_relevant_links("https://huggingface.co"))

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 9 relevant links
## Landing Page:

Hugging Face – The AI community building the future.

Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
NEW
GGML and llama.cpp join Hugging Face 🔥
Try HuggingChat Omni – Chat with AI 💬
Get started with Inference in seconds 🚀
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 2M+ models
Trending on
this week
Models
Qwen/Qwen3.5-397B-A17B
Updated
about 22 hours ago
•
390k
•
973
zai-org/GLM-5
Updated
11 days ago
•
180k
•
1.5k
Nanbeige/Nanbeige4.1-3B
Updated
2 days ago
•
202k
•
765
nvidia/personaplex-7b-v1
Updated
9 days ago
•
539k
•
2.18k
MiniMaxAI/MiniMax-M2.5
Updated
8 days ago
•
224k
•
894
Browse 2M+ models
Spaces
Running
on
Zero
Featured
1.57k
Qwen Image Multiple Angles 3D Camera
🎥
1.57k
Change the camera angle of a photo wi

In [36]:
#brochure_system_prompt = """
#You are an assistant that analyzes the contents of several relevant pages from a company website
#and creates a short brochure about the company for prospective customers, investors and recruits.
#Respond in markdown without code blocks.
#Include details of company culture, customers and careers/jobs if you have the information.
#"""

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

brochure_system_prompt = """
 You are an assistant that analyzes the contents of several relevant pages from a company website
 and creates a short, humorous, entertaining, witty brochure about the company for prospective customers, investors and recruits.
 Respond in markdown without code blocks.
 Include details of company culture, customers and careers/jobs if you have the information.
 """


In [24]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"""
You are looking at a company called: {company_name}
Here are the contents of its landing page and other relevant pages;
use this information to build a short brochure of the company in markdown without code blocks.\n\n
"""
    user_prompt += fetch_page_and_all_relevant_links(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [30]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 12 relevant links


'\nYou are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages;\nuse this information to build a short brochure of the company in markdown without code blocks.\n\n\n## Landing Page:\n\nHugging Face – The AI community building the future.\n\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nNEW\nGGML and llama.cpp join Hugging Face 🔥\nTry HuggingChat Omni – Chat with AI 💬\nGet started with Inference in seconds 🚀\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 2M+ models\nTrending on\nthis week\nModels\nQwen/Qwen3.5-397B-A17B\nUpdated\nabout 22 hours ago\n•\n390k\n•\n973\nzai-org/GLM-5\nUpdated\n11 days ago\n•\n180k\n•\n1.5k\nNanbeige/Nanbeige4.1-3B\nUpdated\n2 days ago\n•\n202k\n•\n765\nnvidia/personaplex-7b-v1\nUpdated\n9 days ago\n•\n539k\n•\n2.18k\nMiniMaxAI/Mini

In [31]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [32]:
create_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 9 relevant links


# Hugging Face: Building the Future of AI Together

---

## About Hugging Face

Hugging Face is a vibrant AI community and collaboration platform dedicated to democratizing machine learning. It serves as a central hub where machine learning engineers, researchers, data scientists, and enthusiasts come together to create, share, and explore open-source models, datasets, and applications across multiple AI modalities including text, image, video, audio, and 3D.

With over 2 million models, 500k datasets, and 1 million+ applications available, Hugging Face has become the go-to place for accelerating AI projects, from research prototypes to enterprise-grade solutions.

---

## Our Mission

- **Democratize Machine Learning:** Make cutting-edge AI technologies accessible to everyone.
- **Open Collaboration:** Build and share ML models and datasets as a community.
- **Build Ethical AI:** Foster transparency and ethical standards in AI development.
- **Accelerate Innovation:** Provide tools and infrastructure that help you move faster with your AI projects.

---

## What We Offer

### Hugging Face Hub
- Host and collaborate on unlimited public models, datasets, and applications.
- Access a fast-growing open source library with the most popular ML models such as DistilBERT.
- Explore powerful AI applications like Qwen3-TTS speech generation and advanced image/video generation models.

### Spaces  
- Run AI demo apps and interactive machine learning experiments directly on the platform.
- Explore cutting-edge use cases like 3D camera angle changes, video generation from images, and speech synthesis.

### Enterprise Solutions  
- Custom AI solutions and compute infrastructure designed for scalability and reliability.
- Flexible pricing plans tailored to enterprise needs.

### Community & Learning  
- Engage with a passionate community of 83,000+ AI professionals and enthusiasts.
- Access papers, detailed tutorials, and articles on the latest AI research and trends.
- Join initiatives like Hugging Face Fundamentals for learning and development.

---

## Our Culture

At Hugging Face, we believe in open collaboration, transparency, and inclusivity. The culture is community-driven — everyone is encouraged to contribute, share knowledge, and push the boundaries of what's possible in machine learning. We are passionate about creating tools that not only empower engineers but also uplift the entire AI ecosystem with ethics and responsibility.

Join us if you want to contribute to real-world AI breakthroughs in a supportive, creative, and fast-paced environment.

---

## Join the Hugging Face Community

- **For ML Engineers and Developers:**
  Share your models, build your portfolio, and collaborate with thousands of contributors worldwide.
  
- **For Researchers and Scientists:**
  Access state-of-the-art datasets and experiment with powerful tools to accelerate discovery.
  
- **For Enterprises:**
  Leverage our platform and compute infrastructure to deploy AI at scale with confidence.
  
- **For Learners and Enthusiasts:**
  Discover tutorials, papers, and learn from a community that’s shaping the future of AI.

---

## Careers at Hugging Face

We are growing quickly and looking for passionate individuals focused on machine learning, engineering, and AI ethics. If you want to be part of an open-source driven company that values every code commit and contribution, consider joining our team!

- Collaborative and inclusive work environment
- Opportunities to work on impactful AI products
- A culture that nurtures learning and personal growth

Check out our careers page to explore open roles and start your journey with Hugging Face.

---

## Get Started Today

Visit [huggingface.co](https://huggingface.co) and:

- Explore 2M+ ML models and 500k datasets
- Try interactive AI apps on Spaces
- Join the community and contribute your own work
- Accelerate your AI projects with open source tools and paid compute

---

## Contact & Press

For media inquiries or partnerships, you can contact the Hugging Face team directly via their website.

---

**Hugging Face — The AI community building the future, one commit at a time.**  
*Join us to shape an open and ethical AI world.*

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [33]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)

In [34]:
stream_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 10 relevant links


# Hugging Face Brochure

---

## About Hugging Face

Hugging Face is a vibrant AI community dedicated to building the future of machine learning through open collaboration. As the central platform where machine learning engineers, scientists, and AI enthusiasts come together, Hugging Face empowers users worldwide to share, discover, and experiment with an extensive range of machine learning models, datasets, and applications.

With over 2 million models, 500,000+ datasets, and 1 million+ applications, Hugging Face offers an unparalleled ecosystem where users can accelerate their AI development, explore every modality — including text, image, video, audio, and even 3D — and build their portfolios to showcase their work to the global ML community.

---

## What We Offer

### The Hugging Face Hub
- **Model Hosting & Sharing:** Host unlimited public models and access cutting-edge pre-trained models.
- **Datasets Library:** Discover and collaborate on a wide variety of datasets essential for ML research and development.
- **Spaces:** Deploy and share ML-powered applications and demos with ease.
- **Open-Source Stack:** Access a complete ML open-source stack that accelerates your model development and deployment.

### Community & Collaboration
- A fast-growing, engaged community driven by collaboration and knowledge-sharing.
- Share your work, learn from others, and connect with industry leaders globally.
- Participate in a culture that values openness, ethics, and innovation in AI.

---

## Enterprise Solutions

Hugging Face offers robust enterprise solutions designed for organizations that require scale, security, and enhanced collaboration:

- **Advanced Security Features:** Single Sign-On (SSO), audit logs, centralized token management, granular access controls.
- **Flexible Management:** Resource groups for role-based access, dataset viewers for private data collaboration.
- **Scalability:** Advanced compute options including ZeroGPU with quota boosts tailored for organizational use.
- **Comprehensive Analytics:** Monitor repository usage and manage spending with intuitive dashboards.
- **Private Storage:** Additional private storage space with flexible purchase options.
- **Dedicated Support:** Enterprise-grade support tailored to your organization's needs.

Plans start as affordable as $20/user/month, with flexible enterprise contracts available.

---

## Company Culture & Values

- **Open & Ethical AI:** Committed to building an AI future that is transparent, ethical, and accessible to all.
- **Community-Driven:** Fosters collaboration among researchers, practitioners, and developers worldwide.
- **Innovation at the Edge:** Supported by a talented science team pushing the boundaries of AI technology.
- **Learning & Growth:** Encourages sharing knowledge and building professional ML profiles.

---

## Careers & Opportunities

Join Hugging Face and be part of the AI revolution! Whether you are an ML engineer, data scientist, researcher, or developer, Hugging Face offers a dynamic environment to innovate and grow alongside some of the most talented minds in the field.

Check their Careers page regularly for openings, and if you share their passion for open source, collaboration, and ethical AI, Hugging Face could be your next inspiring workplace.

---

## Get Started

- Explore models, datasets, and apps at https://huggingface.co
- Sign up and build your ML profile in minutes
- Join the community on GitHub, Twitter, LinkedIn, and Discord

---

## Brand & Visual Identity

Hugging Face's vibrant brand is characterized by a friendly, modern aesthetic featuring bright colors such as #FFD21E and #FF9D00. Their recognizable logos and design assets are available for community and partner use, reinforcing a consistent and approachable visual identity.

---

For more information, visit [Hugging Face](https://huggingface.co)

---

**Hugging Face — The AI community building the future.**

In [37]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 11 relevant links


# Welcome to Hugging Face  
*The AI community that's hugging the future — literally!*

---

## Who Are We?

Imagine a place where machine learning models, datasets, and AI applications go on playdates, collaborate on projects, and develop next-level intelligence together. Welcome to Hugging Face — your friendly neighborhood powerhouse building the future of AI with a big smile and a bigger community!

---

## What We Do — In a Nutshell (or a Hug)

- **Models, Models, Models!**  
  Over **2 million** AI models to explore, tweak, and deploy. From text to images, video, audio, even 3D — we cover all modalities with open arms.

- **Datasets Galore:**  
  Hundreds of thousands of datasets, constantly refreshed and ready to fuel your AI projects. Eyes on accuracy, brains on reason — datasets updated just hours ago!

- **Spaces - Your AI Playground:**  
  Host or browse over 1 million AI applications—think of it as the coolest hackathon that never ends. Demo your stuff or get inspired by others making magic with AI every day.

- **Community Collaboration:**  
  Hugging Face isn’t just a platform — it’s a thriving **community** of 83,000+ AI enthusiasts, researchers, and developers working together to push the envelope of open-source AI.

---

## Our Culture: Hug It Out!

- **Open Source & Open Hearts:**  
  We're powered by transparency, collaboration, and the relentless spirit of sharing. No gatekeeping here—just a big welcoming door.

- **Fast, Friendly, Forward:**  
  Want to get started with AI? Inference on your models in *seconds*, not years! We love building accessible tools that accelerate innovations at lightning speed.

- **Diverse & Dynamic:**  
  Whether you’re an AI veteran, a curious newbie, or a visionary investor, you’ll fit right in. Our culture thrives on diversity, creativity, and a shared mission to democratize AI.

---

## For Our Customers & Users

If you’re a data scientist, developer, researcher, or just an AI aficionado:

- Tap into a universe of over 2 million models.
- Access half a million datasets — more data, more power.
- Spin up your AI app quickly in your own Spaces.
- Join conversations with passionate experts and innovators.
- Use enterprise-grade solutions tailored to your needs.

---

## Join the Hug!

- **Careers:**  
  Love making AI smarter, more inclusive, and downright fun? Hugging Face is hiring talents who want to innovate, collaborate, and help build open AI ecosystems for everyone. 

- **Roles:** From research whizzes, software engineers, community managers, to sales and business development pros — there’s a hug-shaped spot just for you.

- **Why Work Here:**  
  Flexible work styles, vibrant team spirit, and the chance to impact millions of AI users worldwide. Plus, bragging rights about being part of the coolest AI family on the planet.

---

## Fun Facts & Highlights

- Recently welcomed **GGML and llama.cpp** to the HF family — fans of cheeky AI nicknames rejoice!  
- Try **HuggingChat Omni**, our friendly chat with AI that’s smarter than your average bot.  
- Trending models include a whopping **Qwen3.5-397B-A17B** (yes, that’s a lot of numbers and even more brainpower).

---

## Ready to Get Hugged by AI?

Swing by [huggingface.co](https://huggingface.co) — browse models, join the community, or build your own AI masterpiece.  

Because at Hugging Face, the future isn’t just built by AI—it's *hugged* into being. 🤗🚀  

---

## Contact & Stay Connected

- Website: [https://huggingface.co](https://huggingface.co)  
- Community: 83,000+ strong and growing  
- Always open to AI dreamers, doers, and rebels.

---

Hugging Face — *where machines learn, humans share, and the future feels like a big warm hug.*

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>